In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pickle
import cv2
import tqdm
from pathlib import Path
import imageio as iio
import imgaug.augmenters as ia
import imgaug
import numpy as np
from PIL import Image, ImageOps

In [ ]:
data_dir = '/content/drive/MyDrive/cs406/TRECVID/Dataset'

class_names = sorted(os.listdir(data_dir))
print("Class Names:", class_names)
print("Number of classes: ", len(class_names))

Class Names: ['calloused_hands_baseball_field', 'calloused_hands_car', 'calloused_hands_diner', 'calloused_hands_josh_s_house', 'calloused_hands_rabbi_s_house', 'calloused_hands_school', 'calloused_hands_synagogue', 'chained_for_life_garden', 'chained_for_life_hospital', 'chained_for_life_hotel', 'liberty_kid_carnival', 'liberty_kid_house1', 'liberty_kid_house2', 'liberty_kid_prison', 'liberty_kid_statue_of_liberty', 'like_me_diner', 'like_me_gas_station', 'like_me_motel', 'little_rock_caravan', 'little_rock_fun_place', 'little_rock_gallery', 'little_rock_house1', 'little_rock_house2', 'little_rock_motel', 'little_rock_shop', 'losing_ground_home1', 'losing_ground_home2', 'losing_ground_neighbourhood', 'losing_ground_rental_house', 'losing_ground_rooftop', 'losing_ground_university-workplace']
Number of classes:  31


In [ ]:
source = '/content/drive/MyDrive/cs406/TRECVID/Dataset'
target = '/content/drive/MyDrive/cs406/TRECVID/datadir'

In [ ]:
iter = ['/' + class_name for class_name in class_names]
print(iter)

['/calloused_hands_baseball_field', '/calloused_hands_car', '/calloused_hands_diner', '/calloused_hands_josh_s_house', '/calloused_hands_rabbi_s_house', '/calloused_hands_school', '/calloused_hands_synagogue', '/chained_for_life_garden', '/chained_for_life_hospital', '/chained_for_life_hotel', '/liberty_kid_carnival', '/liberty_kid_house1', '/liberty_kid_house2', '/liberty_kid_prison', '/liberty_kid_statue_of_liberty', '/like_me_diner', '/like_me_gas_station', '/like_me_motel', '/little_rock_caravan', '/little_rock_fun_place', '/little_rock_gallery', '/little_rock_house1', '/little_rock_house2', '/little_rock_motel', '/little_rock_shop', '/losing_ground_home1', '/losing_ground_home2', '/losing_ground_neighbourhood', '/losing_ground_rental_house', '/losing_ground_rooftop', '/losing_ground_university-workplace']


In [ ]:
# Creating folder
for class_name in class_names:
  try:
    os.mkdir(target + '/' + class_name)
    print(f'Make dir {class_name} successful!')
  except:
    pass

## Single transform

In [ ]:
def ChannelShuffle(source, target):
    for i in iter:
        count = 0
        for file in Path(source + "/" + i).iterdir():
            img = iio.imread(file)
            count += 1
            aug = ia.ChannelShuffle(1)
            imgnew = aug.augment_image(img)
            imgnew = cv2.cvtColor(imgnew, cv2.COLOR_RGBA2RGB)
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
            iio.imwrite(target + '/' + i + '/' + str(count) + "_channel_shift" + '.JPG', imgnew)
            iio.imwrite(target + '/' + i + '/' + str(count) + '.JPG', img)

In [ ]:
def flipImages(source, target):
    for i in iter:
        count = 0
        for j in os.listdir(source + i):
            count += 1
            img = Image.open(source + i + '/'+ j)
            im_flip = ImageOps.flip(img)
            im_mirror = ImageOps.mirror(img)
            if im_flip.mode == 'RGBA':
              im_flip = im_flip.convert('RGB')
            if im_mirror.mode == 'RGBA':
              im_mirror = im_mirror.convert('RGB')
            im_flip.save(target + '/' + i + '/' + str(count) + "_flipped" + ".JPG")
            im_mirror.save(target + '/' + i + '/' + str(count) + "_mirrored" + ".JPG")
            img.close()

In [ ]:
def GaussianBlur(source, target):
    for i in iter:
        count = 0
        for file in Path(source + "/" + i).iterdir():
            img = iio.imread(file)
            count += 1
            aug = ia.GaussianBlur()
            imgnew = aug.augment_image(img)
            imgnew = cv2.cvtColor(imgnew, cv2.COLOR_RGBA2RGB)
            iio.imwrite(target + '/' + i + '/' + str(count) + "_gaussian_blur" + '.JPG', imgnew)
            aug = ia.Sharpen(alpha=(0,1.0), lightness=(.75, 1.5))
            imgnew = aug.augment_image(img)
            imgnew = cv2.cvtColor(imgnew, cv2.COLOR_RGBA2RGB)
            iio.imwrite(target + '/' + i + '/' + str(count) + "_sharpen" + '.JPG', imgnew)
            aug = ia.AdditiveGaussianNoise(loc=0, scale=(0.0,0.05*225), per_channel=0.5)
            imgnew = aug.augment_image(img)
            imgnew = cv2.cvtColor(imgnew, cv2.COLOR_RGBA2RGB)
            iio.imwrite(target + '/' + i + '/' + str(count) + "_AdditiveNoise" + '.JPG', imgnew)

In [ ]:
def GaussianNoise(source, target):
    for i in iter:
        count = 0
        for file in Path(source + "/" + i).iterdir():
            img = iio.imread(file)
            count += 1
            aug = ia.AdditiveGaussianNoise(scale=(0, 0.4*255))
            imgnew = aug.augment_image(img)
            iio.imwrite(target + '/' + i + '/' + str(count) + "_gaussian_noise" + '.JPG', imgnew)
            iio.imwrite(target + '/' + i + '/' + str(count) + '.JPG', img)

In [ ]:
def rotateImages(source, target, angle):
    for i in iter:
        count = 0
        for j in os.listdir(source + i):
            count += 1
            img = Image.open(source + i + '/'+ j)
            im_rotate = img.rotate(angle)
            if im_rotate.mode == 'RGBA':
              im_rotate = im_rotate.convert('RGB')
            im_rotate.save(target + i + "/" + str(count) + "_" + str(angle) + ".JPG")
            img.close()

In [ ]:
def shear(source, target):
    for i in iter:
        count = 0
        for j in os.listdir(source + i):
            with Image(filename = (source + '/' + i + '/' + j)) as image:
                with image.clone() as shear:
                    shear.shear()

In [ ]:
ChannelShuffle(source, target)

<ipython-input-62-ef8f0a112287>:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = iio.imread(file)


In [ ]:
flipImages(source, target)

In [ ]:
GaussianBlur(source, target)

<ipython-input-49-1ac7a8d7e20a>:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = iio.imread(file)


In [ ]:
for i in range(0, 360, 5):
  rotateImages(source, target, i)

## Sequence transform

### Multiple

In [ ]:
seq = ia.Sequential([
    ia.Fliplr(0.5), # horizontal flips
    ia.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    ia.Sometimes(
        0.5,
        ia.GaussianBlur(sigma=(0, 0.5))
    ),
    # Strengthen or weaken the contrast in each image.
    #ia.LinearContrast((0.75, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    ia.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    #ia.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    ia.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) # apply augmenters in random order

In [ ]:
def sequence(source, target):
    for i in iter:
        count = 0
        for file in Path(source + "/" + i).iterdir():
            img = iio.imread(file)
            count += 1
            for k in range(500):
                imgnew = seq.augment_image(img);
                imgnew = cv2.cvtColor(imgnew, cv2.COLOR_RGBA2RGB)
                iio.imwrite(target + '/' + i + '/' + str(count) + "_v" + str(k) + '.JPG', imgnew)

In [ ]:
sequence(source, target)

<ipython-input-67-814d2c9918fa>:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = iio.imread(file)


FileNotFoundError: ignored

### Augmentation

In [ ]:
sometimes = lambda aug: ia.Sometimes(0.5, aug)

# Define our sequence of augmentation steps that will be applied to every image.
seq = ia.Sequential(
    [
        #
        # Apply the following augmenters to most images.
        #
        #ia.Fliplr(0.5), # horizontally flip 50% of all images
        #ia.Flipud(0.2), # vertically flip 20% of all images

        # crop some of the images by 0-10% of their height/width
        #sometimes(ia.Crop(percent=(0, 0.1))),

        # Apply affine transformations to some of the images
        # - scale to 80-120% of image height/width (each axis independently)
        # - translate by -20 to +20 relative to height/width (per axis)
        # - rotate by -45 to +45 degrees
        # - shear by -16 to +16 degrees
        # - order: use nearest neighbour or bilinear interpolation (fast)
        # - mode: use any available mode to fill newly created pixels
        #         see API or scikit-image for which modes are available
        # - cval: if the mode is constant, then use a random brightness
        #         for the newly created pixels (e.g. sometimes black,
        #         sometimes white)
        sometimes(ia.Affine(
            #scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            #rotate=(-45, 45),
            shear=(-16, 16),
            order=[0, 1],
            cval=(0, 255),
            mode=imgaug.ALL
        )),

        #
        # Execute 0 to 5 of the following (less important) augmenters per
        # image. Don't execute all of them, as that would often be way too
        # strong.
        #
        ia.SomeOf((0, 5),
            [
                # Convert some images into their superpixel representation,
                # sample between 20 and 200 superpixels per image, but do
                # not replace all superpixels with their average, only
                # some of them (p_replace).
                sometimes(
                    ia.Superpixels(
                        p_replace=(0, 1.0),
                        n_segments=(20, 200)
                    )
                ),

                # Blur each image with varying strength using
                # gaussian blur (sigma between 0 and 3.0),
                # average/uniform blur (kernel size between 2x2 and 7x7)
                # median blur (kernel size between 3x3 and 11x11).
                ia.OneOf([
                    ia.GaussianBlur((0, 3.0)),
                    ia.AverageBlur(k=(2, 7)),
                    ia.MedianBlur(k=(3, 11)),
                ]),

                # Sharpen each image, overlay the result with the original
                # image using an alpha between 0 (no sharpening) and 1
                # (full sharpening effect).
                ia.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),

                # Same as sharpen, but for an embossing effect.
                ia.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),

                # Search in some images either for all edges or for
                # directed edges. These edges are then marked in a black
                # and white image and overlayed with the original image
                # using an alpha of 0 to 0.7.
                sometimes(ia.OneOf([
                    ia.EdgeDetect(alpha=(0, 0.7)),
                    ia.DirectedEdgeDetect(
                        alpha=(0, 0.7), direction=(0.0, 1.0)
                    ),
                ])),

                # Add gaussian noise to some images.
                # In 50% of these cases, the noise is randomly sampled per
                # channel and pixel.
                # In the other 50% of all cases it is sampled once per
                # pixel (i.e. brightness change).
                ia.AdditiveGaussianNoise(
                    loc=0, scale=(0.0, 0.05*255), per_channel=0.5
                ),

                # Either drop randomly 1 to 10% of all pixels (i.e. set
                # them to black) or drop them on an image with 2-5% percent
                # of the original size, leading to large dropped
                # rectangles.
                ia.OneOf([
                    ia.Dropout((0.01, 0.1), per_channel=0.5),
                    ia.CoarseDropout(
                        (0.03, 0.15), size_percent=(0.02, 0.05),
                        per_channel=0.2
                    ),
                ]),

                # Invert each image's channel with 5% probability.
                # This sets each pixel value v to 255-v.
                #ia.Invert(0.05, per_channel=True), # invert color channels

                # Add a value of -10 to 10 to each pixel.
                #ia.Add((-10, 10), per_channel=0.5),

                # Change brightness of images (50-150% of original value).
                ia.Multiply((0.5, 1.5), per_channel=0.5),

                # Improve or worsen the contrast of images.
                ia.LinearContrast((0.5, 2.0), per_channel=0.5),

                # Convert each image to grayscale and then overlay the
                # result with the original with random alpha. I.e. remove
                # colors with varying strengths.
                #ia.Grayscale(alpha=(0.0, 1.0)),

                # In some images move pixels locally around (with random
                # strengths).
                sometimes(
                    ia.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)
                ),

                # In some images distort local areas with varying strength.
                sometimes(ia.PiecewiseAffine(scale=(0.01, 0.05)))
            ],
            # do all of the above augmentations in random order
            random_order=True
        )
    ],
    # do all of the above augmentations in random order
    random_order=True)
def sequence():
    for i in iter:
        count = 0
        for file in Path(source + "/" + i).iterdir():
            img = iio.imread(file)
            count += 1
            for k in range(500):
                imgnew = seq.augment_image(img);
                iio.imwrite(target + '/' + i + '/' + str(count) + "_v" + str(k) + '.JPG', imgnew)
            iio.imwrite(target + '/' + i + '/' + str(count) + '.JPG', img)

In [ ]:
def sequence():
    for i in iter:
        count = 0
        for file in Path(source + "/" + i).iterdir():
            img = iio.imread(file)
            count += 1
            for k in range(500):
                imgnew = seq.augment_image(img);
                imgnew = cv2.cvtColor(imgnew, cv2.COLOR_RGBA2RGB)
                iio.imwrite(target + '/' + i + '/' + str(count) + "_v" + str(k) + '.JPG', imgnew)
            iio.imwrite(target + '/' + i + '/' + str(count) + '.JPG', img)

In [ ]:
sequence()

<ipython-input-58-4644c3903c4e>:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = iio.imread(file)


AttributeError: ignored